In [2]:
!pip install -r requirements.txt

In [4]:
import numpy as np
from clustimage import Clustimage

from matplotlib import image

import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

import os
import sys
import cv2

2023-05-09 18:25:58.853675: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-09 18:25:59.981875: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-05-09 18:25:59.982049: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/

In [6]:
#Prepare the Data

#in coco format
paths=['downloads/SoccerPass/SoccerPass-COCO/train.json']

import json
import pandas as pd
from pandas import json_normalize

dfs= []
for path in paths:
    # Load the COCO data from the JSON file
    with open(path) as f:
        coco_data = json.load(f)

    # Normalize the data into a DataFrame
    images_df = json_normalize(coco_data['images'])
    annotations_df = json_normalize(coco_data['annotations'])

    images_df=images_df.drop(columns=['license', 'flickr_url','coco_url', 'date_captured'])
    annotations_df=annotations_df.drop(columns=['segmentation', 'area','attributes.occluded','attributes.rotation','iscrowd'])
    annotations_df[['x', 'y', 'width', 'height']]=pd.DataFrame(annotations_df['bbox'].to_list())

    annotations_df['x'] = annotations_df['x'] + annotations_df['width']//2
    annotations_df['y'] = annotations_df['y'] + annotations_df['height']//2


    merged_df = pd.merge(annotations_df, images_df, right_on='id',left_on="image_id", how='left')

    merged_df['x'] = merged_df['x'] /merged_df['width_y']
    merged_df['width_x'] = merged_df['width_x'] /merged_df['width_y']
    merged_df['y'] = merged_df['y'] /merged_df['height_y']
    merged_df['height_x'] = merged_df['height_x'] /merged_df['height_y']


    merged_df['new_id'] = merged_df.groupby('image_id').cumcount() + 1
    merged_df=merged_df.drop(columns=['id_x', 'image_id', 'bbox', 'id_y', 'width_y', 'height_y'])

    new_columns = {'category_id': 'class',
                'attributes.Team': 'team',
                'attributes.Ball_Possesion': 'Ball_Possesion',
                'attributes.Ball_Reciever': 'Ball_Reciever',
                'width_x': 'w',
                'height_x': 'h',
                'file_name': 'image_name',
                'new_id': 'OID'}

    merged_df = merged_df.rename(columns=new_columns)


    # merged_df.to_csv("mer.csv",index=False)

    dfs.append(merged_df)

label_df=pd.concat(dfs)


In [7]:
label_df

,class,Ball_Possesion,Ball_Reciever,team,x,y,w,h,image_name,OID
0,1,0,0,1,0.982031,0.319444,0.030469,0.123611,image_vics_1342.png,1
1,2,NaN,NaN,NaN,0.337500,0.595833,0.011719,0.020833,image_vics_1342.png,2
2,1,0,0,0,0.735156,0.084722,0.020313,0.062500,image_vics_1342.png,3
3,1,0,0,0,0.200781,0.130556,0.018750,0.065278,image_vics_1342.png,4
4,1,0,0,1,0.586719,0.391667,0.030469,0.119444,image_vics_1342.png,5
...,...,...,...,...,...,...,...,...,...,...
16306,1,0,0,0,0.326562,0.472222,0.022656,0.094444,image_vimp_689.png,14
16307,1,0,0,2,0.421328,0.358333,0.017734,0.094444,image_vimp_689.png,15
16308,1,0,1,0,0.237500,0.223611,0.025000,0.059722,image_vimp_689.png,16
16309,1,0,0,0,0.327344,0.397222,0.018750,0.070833,image_vimp_689.png,17


In [9]:
def pre_process_crop(image, bbox):
    H = image.shape[0]
    W = image.shape[1]

    x = int(bbox[0]*W)
    y = int(bbox[1]*H)
    w = int(bbox[2]*W)
    h = int(bbox[3]*H)
    
    #Extracting Crop from given image with bbox specs
    crop = image[y:y+h, x:x+w]
    crop = cv2.resize(crop, (20,40))

    #Applying filters to the crop
    crop = cv2.medianBlur(crop,3)
    crop = cv2.bilateralFilter(crop,5,30,30)
    

    return crop

In [10]:

def calc_acc(pred_labels, true_labels):
  n = len(true_labels)
  count=0
  for ix in range(n):
    if (true_labels[ix]==pred_labels[ix]):
      count+=1
  
  return count/n


In [11]:
def formatPred(pred):
  y = np.unique(pred, return_counts=True)
  z = sorted(zip(y[1], y[0]), reverse=True)
  res = pred.copy()
  i=0
  for tup in z:
    idx_list = np.where(pred == tup[1])[0]
    
    # print(tup, idx_list)
    for idx in idx_list:
      res[idx] = i
    i+=1

  return fin


In [14]:
paths = ['downloads/SoccerPass/SoccerPass-COCO/images/train']

In [17]:
def cluster(yolo_df):
  total_boxes = 0
  total_misclassified = 0
  
  groups = yolo_df.groupby("image_name")


  # loop through each group and apply the code snippet to each image
  for image_name, group in groups:
      image_path = None
      for path in paths:
          temp_path = os.path.join(path, image_name)
          if os.path.exists(temp_path):
              image_path = temp_path
              break
      
      if image_path is None:
          raise ValueError(f"Image file not found: {image_name}")

      img = image.imread(image_path)
      # plt.imshow(img)
      # plt.show()

      players_info = group[group['class'] == 1]

      true_labels = np.array(players_info['team'].tolist(), dtype=np.int32)
      num_cats = len(pd.unique(players_info['team'])) 

      player_boxes = []
      
      for index, row in players_info.iterrows():
        bbox = [row['x']-(row['w']/2), row['y']-(row['h']/2), row['w'], row['h']]
        crop = pre_process_crop(img, bbox)
        player_boxes.append(crop)

      player_boxes_reshaped = np.array(player_boxes).reshape(len(player_boxes),-1)
      player_boxes_reshaped = player_boxes_reshaped/255.0

      cl = Clustimage(method='pca',
                    embedding='tsne',
                    grayscale=False,
                    dim=(20,40),
                    params_pca={'n_components':0.95},
                    verbose=60)

      results = cl.fit_transform(player_boxes_reshaped,
                                cluster='agglomerative',
                                evaluate='silhouette',
                                metric='euclidean',
                                linkage='ward',
                                min_clust=num_cats,
                                max_clust=num_cats,
                                cluster_space='high')

      pred_labels = results['labels']
      
      res = formatPred(pred_labels)

      # true_labels = np.array(true_labels[0])
      if (calc_acc(res[0], true_labels) > calc_acc(res[1], true_labels)):
        acc = calc_acc(res[0], true_labels)
        preds = res[0]
      else:
        acc = calc_acc(res[1], true_labels)
        preds = res[1]

      total_boxes += group.shape[0]
      total_misclassified += round(group.shape[0]*(1-acc))

      i = 0
      for index, row in players_info.iterrows():
        if(row['class'] == 2):
          continue
        yolo_df.loc[index, 'pred_team'] = int(preds[i])
        i+=1
      print("name: {}, accuracy: {}, preds: {}, misclassified: {}, total boxes: {}".format(image_name, acc, preds, round(players_info.shape[0]*(1-acc)), players_info.shape[0]))

  print("total_boxes: {} ,  total_misclassified: {}".format(total_boxes, total_misclassified))
  return yolo_df




In [18]:
clustered_df = cluster(label_df)

[pca] >Column labels are auto-completed.
[pca] >The PCA reduction is performed to capture [95.0%] explained variance using the [2400] columns of the input data.
[pca] >Fit using PCA.
[pca] >Compute loadings and PCs.
[pca] >Compute explained variance.
[pca] >Number of components is [8] that covers the [95.00%] explained variance.
[pca] >The PCA reduction is performed on the [2400] columns of the input dataframe.
[pca] >Fit using PCA.
[pca] >Compute loadings and PCs.


[clustimage] >INFO> Saving data in memory.
[clustimage] >INFO> Fit with method=[agglomerative], metric=[euclidean], linkage=[ward]
[clustimage] >INFO> Evaluate using silhouette.
[clusteval] >INFO: 100%|████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 670.23it/s]
[clustimage] >DEBUG> [[2 2 1 1 1 2 2 2 2 1 2]]
[clustimage] >DEBUG> Iloc: [ True]
[clustimage] >DEBUG> silscores: [0.45013442]
[clustimage] >DEBUG> sillclust: [2.]
[clustimage] >DEBUG> clustlabx: [[2 2 1 1 1 2 2 2 2 1 2]]
[clustimage] >INFO> Compute dendrogram threshold.
[clustimage] >INFO> Optimal number clusters detected: [2].
[clustimage] >INFO> Fin.


name: image_prmw_1239.png, accuracy: 1.0, preds: [1 1 0 0 0 1 1 1 1 0 1], misclassified: 0, total boxes: 11
[pca] >Column labels are auto-completed.
[pca] >The PCA reduction is performed to capture [95.0%] explained variance using the [2400] columns of the input data.
[pca] >Fit using PCA.
[pca] >Compute loadings and PCs.
[pca] >Compute explained variance.
[pca] >Number of components is [12] that covers the [95.00%] explained variance.
[pca] >The PCA reduction is performed on the [2400] columns of the input dataframe.
[pca] >Fit using PCA.
[pca] >Compute loadings and PCs.


[clustimage] >INFO> Saving data in memory.
[clustimage] >INFO> Fit with method=[agglomerative], metric=[euclidean], linkage=[ward]
[clustimage] >INFO> Evaluate using silhouette.
[clusteval] >INFO: 100%|████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 187.95it/s]
[clustimage] >DEBUG> [[2 1 3 2 1 3 3 2 1 3 1 1 3 1 1 1]]
[clustimage] >DEBUG> Iloc: [ True]
[clustimage] >DEBUG> silscores: [0.23475963]
[clustimage] >DEBUG> sillclust: [3.]
[clustimage] >DEBUG> clustlabx: [[2 1 3 2 1 3 3 2 1 3 1 1 3 1 1 1]]
[clustimage] >INFO> Compute dendrogram threshold.
[clustimage] >INFO> Optimal number clusters detected: [3].
[clustimage] >INFO> Fin.


name: image_prmw_1241.png, accuracy: 0.75, preds: [2 1 0 2 1 0 0 2 1 0 1 1 0 1 1 1], misclassified: 4, total boxes: 16
[pca] >Column labels are auto-completed.
[pca] >The PCA reduction is performed to capture [95.0%] explained variance using the [2400] columns of the input data.
[pca] >Fit using PCA.
[pca] >Compute loadings and PCs.
[pca] >Compute explained variance.
[pca] >Number of components is [11] that covers the [95.00%] explained variance.
[pca] >The PCA reduction is performed on the [2400] columns of the input dataframe.
[pca] >Fit using PCA.
[pca] >Compute loadings and PCs.


...

name: image_vimp_687.png, accuracy: 0.6875, preds: [0 1 0 1 1 1 0 0 2 2 2 1 1 1 0 0], misclassified: 5, total boxes: 16
[pca] >Column labels are auto-completed.
[pca] >The PCA reduction is performed to capture [95.0%] explained variance using the [2400] columns of the input data.
[pca] >Fit using PCA.
[pca] >Compute loadings and PCs.
[pca] >Compute explained variance.
[pca] >Number of components is [10] that covers the [95.00%] explained variance.
[pca] >The PCA reduction is performed on the [2400] columns of the input dataframe.
[pca] >Fit using PCA.
[pca] >Compute loadings and PCs.


[clustimage] >INFO> Saving data in memory.
[clustimage] >INFO> Fit with method=[agglomerative], metric=[euclidean], linkage=[ward]
[clustimage] >INFO> Evaluate using silhouette.
[clusteval] >INFO: 100%|████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 633.39it/s]
[clustimage] >DEBUG> [[1 1 3 2 3 1 2 1 1 3 1 1 1 1 2]]
[clustimage] >DEBUG> Iloc: [ True]
[clustimage] >DEBUG> silscores: [0.33397792]
[clustimage] >DEBUG> sillclust: [3.]
[clustimage] >DEBUG> clustlabx: [[1 1 3 2 3 1 2 1 1 3 1 1 1 1 2]]
[clustimage] >INFO> Compute dendrogram threshold.
[clustimage] >INFO> Optimal number clusters detected: [3].
[clustimage] >INFO> Fin.


name: image_vimp_688.png, accuracy: 0.8666666666666667, preds: [1 1 0 2 0 1 2 1 1 0 1 1 1 1 2], misclassified: 2, total boxes: 15
[pca] >Column labels are auto-completed.
[pca] >The PCA reduction is performed to capture [95.0%] explained variance using the [2400] columns of the input data.
[pca] >Fit using PCA.
[pca] >Compute loadings and PCs.
[pca] >Compute explained variance.
[pca] >Number of components is [11] that covers the [95.00%] explained variance.
[pca] >The PCA reduction is performed on the [2400] columns of the input dataframe.
[pca] >Fit using PCA.
[pca] >Compute loadings and PCs.


[clustimage] >INFO> Saving data in memory.
[clustimage] >INFO> Fit with method=[agglomerative], metric=[euclidean], linkage=[ward]
[clustimage] >INFO> Evaluate using silhouette.
[clusteval] >INFO: 100%|████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 630.91it/s]
[clustimage] >DEBUG> [[2 1 1 1 2 2 1 1 2 1 1 1 1 2 3 2 2]]
[clustimage] >DEBUG> Iloc: [ True]
[clustimage] >DEBUG> silscores: [0.31691897]
[clustimage] >DEBUG> sillclust: [3.]
[clustimage] >DEBUG> clustlabx: [[2 1 1 1 2 2 1 1 2 1 1 1 1 2 3 2 2]]
[clustimage] >INFO> Compute dendrogram threshold.
[clustimage] >INFO> Optimal number clusters detected: [3].
[clustimage] >INFO> Fin.


name: image_vimp_689.png, accuracy: 1.0, preds: [0 1 1 1 0 0 1 1 0 1 1 1 1 0 2 0 0], misclassified: 0, total boxes: 17
total_boxes: 16311 ,  total_misclassified: 2573
